# Assignment-2

1. Implement Backpropagation algorithm to train an ANN of configuration __2x2x1__ to achieve __XOR__ function.
2. Implement Backpropagation algorithm to train an ANN of configuration __3x2x2x1__ to achieve __majority function__ with 3-bit data. Output of the network must be 1 when there are two or more 1’s in the data.

Neural network is defined and while training backpropagation algorithm is used for adjusting the weights so that the weights are adjusted making the neural network best fit to data as possible.

In [21]:
import numpy as np
import numpy.typing as npt

def sigmoid( net_output: npt.ArrayLike ) -> np.ndarray:
        '''
            Sigmoid activation function. 
            Takes an numpy array and computes value using the sigmoid function.
        '''
        output = 1 / ( 1 + np.exp(-net_output))
        return output
    
class Layer:
    '''
        Class that defines each layer of neural network.
        Each layer has a _n_ neurons and some weights connected to it.
        ...
        
        Attributes
        ----------
        wght : ndarray
            weights associated with each neurons.
        layer_no : int
            layer number in the neural network.
        grad : ndarray
            gradients for the weight update.
        delta : ndarray
            error term for the current layer.
        activation: ndarray
            weighted sum of each neuron.
        n_output : ndarray
            output of each neuron.
        
        
        Methods
        -------
        activate_h(input):
            Computes the weighted sum and outputs the activation value after 
            computing the sigmoid value
            for each hidden neuron.
        activate_o(input):
            Computes the weighted sum and outputs the activation value after 
            computing the sigmoid value
            for each outermost neuron.
        
    '''
    def __init__(self, neurons, no_of_wghts, layer_no):
        print(f"Layer {layer_no+1} has {neurons} neurons and each neuron has {no_of_wghts} weights connected to it.")
        self.wght = np.random.normal(size=(neurons, no_of_wghts+1))
        self.layer_no = layer_no+1
        self.grad = np.empty(shape=(neurons, no_of_wghts+1))
        self.delta = np.empty(shape=(1, neurons))
        
    def activate_h(self, input):
        '''
            Computes the weighted sum and outputs the activation value after computing the sigmoid value for hidden neuron.
        '''
        _, col = input.shape
        if self.wght.shape[1] == col:
            self.activation = np.dot(input, self.wght.T)
            self.n_output = sigmoid(self.activation)
            self.n_output = np.c_[self.n_output, np.ones(shape=(self.n_output.shape[0], 1))]
            return self.n_output
        else:
            raise Exception(f"input {input.shape} and weights {self.wght.shape} shape not valid ")
        
    def activate_o(self, input):
        '''
            Computes the weighted sum and outputs the activation value after computing the sigmoid value for output layer. 
        '''
        _, col = input.shape
        if self.wght.shape[1] == col:
            self.activation = np.dot(input,self.wght.T )
            self.n_output = sigmoid(self.activation)
            return self.n_output
        else:
            raise Exception(f"input {input.shape} and weights {self.wght.shape} shape not valid ")
        


In [22]:
class Input(Layer):
    def __init__(self, neurons, layer_no):
        print(f"Input Layer has {neurons} inputs.")
        self.neurons = neurons
        self.layer_no = layer_no+1
    def activate_h(self, input):
        self.n_output = input

In [23]:

class Network:
    '''
        Class defines and initializes the neural network.
        ...
        
        Attributes
        ----------
        layers: list
            list of all layers in the neural network.
        total_layers: int
            number of total layers in the network.
        input : ndarray
            input array for the neural network.
        output : ndarray
            output array for the neural network.

        Methods
        -------
        forward(input): None
            computes the forward propagation phase of the neural network.
        backward(target, learning_rate=1): None
            computes the backward propagation phase of the neural network.
        
    '''
    def __init__(self, *args) -> None:
        self.layers = []
        self.total_layers = len(args)
        self.layers.append(Input(args[0],0))
        for l in range(1, self.total_layers):
            self.layers.append(Layer(args[l], args[l-1], l))
            
    def forward(self, input):
        self.input = input
        input = np.c_[input, np.ones(shape=(input.shape[0],1))]
        self.layers[0].activate_h(input)
        for layer_no in range(1, self.total_layers-1):
            input = self.layers[layer_no].activate_h(input)
        self.output = self.layers[-1].activate_o(input)
        
    def backward(self, target, learning_rate=1):
        diff = target - self.output
        derivative = self.output * ( 1 - self.output)
        delta = diff * derivative
        #output layer weight update
        for d in range(len(self.output)):
            diff = target[d] - self.output[d]
            derivative = self.output[d] * (1 - self.output[d])
            delta = diff * derivative
            grad  = np.dot(delta.T[:,np.newaxis], self.layers[-2].n_output[d][np.newaxis,:])
            self.layers[-1].grad = grad
            self.layers[-1].delta = delta
            #finding the gradients for weight update
            for l in range(self.total_layers-1, 1, -1):
                delta_h = np.dot(self.layers[l].delta, self.layers[l].wght[:, :-1]) # don't need the bias for the previous layers' weight update
                deriv_h = (self.layers[l-1].n_output[d] * ( 1- self.layers[l-1].n_output[d]))[np.newaxis, :-1]
                delta_h = delta_h * deriv_h
                grad_h = np.dot(delta_h.T, self.layers[l-2].n_output[d][np.newaxis, :]) 
                self.layers[l-1].grad = grad_h
                self.layers[l-1].delta = delta_h
            #weight update
            for layer in  range(1, self.total_layers):
                self.layers[layer].wght = self.layers[layer].wght + (learning_rate * self.layers[layer].grad)
            self.forward(self.input)     
            
    def train(self, input, output, learning_rate=1, epochs=1000):
        for e in range(1, epochs+1):
            self.forward(input)
            self.backward(output, learning_rate=learning_rate)             
            print(f"{e} epochs completed.", end="\r")
        print("",end='\n')
    
    def test(self, input):
        self.forward(input)
        print(self.output)

### __Answer__ for __Q. No. 1__: 

The XOR truth table looks like:

| Input      | Output |
| ---------- | ------ |
| 0    ,   0 |   0    |
| 1    ,   0 |   1    |
| 0    ,   1 |   1    |
| 1    ,   1 |   0    |


In [24]:
np.random.seed(42)

net = Network(2, 2, 1)

# test = Layer(2, 2, 1)
input = np.array([[1, 0],
                  [0, 1],
                  [0, 0],
                  [1, 1]])
output = np.array([ [1],
                    [1],
                    [0],
                    [0] ])
net.train(input, output)
net.test(input)


Input Layer has 2 inputs.
Layer 2 has 2 neurons and each neuron has 2 weights connected to it.
Layer 3 has 1 neurons and each neuron has 2 weights connected to it.
1000 epochs completed.
[[0.95257338]
 [0.94988354]
 [0.0559636 ]
 [0.0502493 ]]


### __Answer__ for __Q. No. 2__: 

In [25]:
np.random.seed(42)

net = Network(3, 2, 2, 1)
arr = np.arange(0, 8)
max_len = len(np.binary_repr(arr[-1]))
input = np.array([list(np.binary_repr(x).zfill(max_len)) for x in arr], dtype=int)
output = np.array([[0],
                   [0],
                   [0],
                   [1],
                   [0],
                   [1],
                   [1],
                   [1]])
net.train(input, output)
net.test(input)

Input Layer has 3 inputs.
Layer 2 has 2 neurons and each neuron has 3 weights connected to it.
Layer 3 has 2 neurons and each neuron has 2 weights connected to it.
Layer 4 has 1 neurons and each neuron has 2 weights connected to it.
1000 epochs completed.
[[0.00968035]
 [0.02182572]
 [0.02065927]
 [0.97589547]
 [0.02059313]
 [0.97590901]
 [0.97536999]
 [0.98375435]]
